In [45]:
#链接数据库

import sqlite3
import pandas as pd

conn = sqlite3.connect('HPL.db')

In [46]:
# 解析文件

def extract(filename, data):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # 遍历每一行并解析数据
    # N      :   80000 
    # NB     :     120 
    # PMAP   : Row-major process mapping
    # P      :       4 
    # Q      :       1 
    # PFACT  :    Left 
    # NBMIN  :       1 
    # NDIV   :       2 
    # RFACT  :    Left 
    # BCAST  :   1ring 
    # DEPTH  :       0 
    # SWAP   : Binary-exchange
    # L1     : transposed form
    # U      : transposed form
    # EQUIL  : yes
    # ALIGN  : 8 double precision words
    
    for i in range(len(lines)):
        line = lines[i]
        if "The following parameter values will be used:" in line: 
            i += 2
            N = lines[i].split()[2]
            NB = lines[i + 1].split()[2]
            PMAP = lines[i + 2].split()[2]
            P = lines[i + 3].split()[2]
            Q = lines[i + 4].split()[2]
            PFACT = lines[i + 5].split()[2]
            if len(lines[i + 6].split()) < 3:
                print(i)
                print(lines[i + 6])
            NBMIN = lines[i + 6].split()[2]
            NDIV = lines[i + 7].split()[2]
            RFACT = lines[i + 8].split()[2]
            BCAST = lines[i + 9].split()[2]
            DEPTH = lines[i + 10].split()[2]
            SWAP = lines[i + 11].split()[2]
            L1 = lines[i + 12].split()[2]
            U = lines[i + 13].split()[2]
            EQUIL = lines[i + 14].split()[2]


        elif "T/V                N    NB     P     Q               Time                 Gflops" in line:
            line = lines[i + 2]
            parts = line.split()
            Time = parts[-2]
            Gflops = parts[-1]
            TV = parts[0]
            letters = list(TV)
            DEPTH = letters[2]
            BCAST = letters[3]
            RFACT = letters[4]
            NDIV = letters[5]
            PFACT = letters[6]
            NBMIN = letters[7]
            data.append([PMAP, SWAP, L1, U, EQUIL, DEPTH, BCAST, RFACT, NDIV, PFACT, NBMIN, N, NB, P, Q, Time, Gflops])
    return data

插入4core数据

In [47]:
import os

directory_path = "/home/jiayh_lab/cse12012138/BenchmarkOptimization/logs/4cores/"  # 将此路径替换为您要搜索的目录的路径
file_names = [file for file in os.listdir(directory_path) if file.endswith(".out")]
data = []

for filename in file_names:
    print(filename)
    if 'fail' in filename: continue
    extract(directory_path + filename, data)

# 数据类型转换
columns_to_convert = range(11, 17)
data = [
    [float(value) if index in columns_to_convert else value for index, value in enumerate(row)]
    for row in data
]

# 加核数
data = [[4] + row for row in data]
df = pd.DataFrame(data, columns=['cores', 'PMAP', 'SWAP', 'L1', 'U', 'EQUIL', 'DEPTH', 'BCAST', 'RFACT', 'NDIV', 'PFACT', 'NBMIN', 'N', 'NB', 'P', 'Q', 'Time', 'Gflops'])
df

job.5856.out
job.5889.out
job.6076.out
job.6105.out
job.6111.out
job.6162.out
job.6211.out


,cores,PMAP,SWAP,L1,U,EQUIL,DEPTH,BCAST,RFACT,NDIV,PFACT,NBMIN,N,NB,P,Q,Time,Gflops
0,4,Row-major,Binary-exchange,transposed,transposed,no,0,0,L,2,L,1,80000.0,120.0,4.0,1.0,1430.09,238.69
1,4,Row-major,Binary-exchange,transposed,transposed,yes,0,0,L,2,L,1,80000.0,120.0,4.0,1.0,1531.65,222.86
2,4,Row-major,Spread-roll,transposed,transposed,no,0,0,L,2,L,1,80000.0,120.0,4.0,1.0,1495.00,228.32
3,4,Row-major,Spread-roll,transposed,transposed,yes,0,0,L,2,L,1,80000.0,120.0,4.0,1.0,1489.86,229.11
4,4,Row-major,Spread-roll,transposed,transposed,no,0,0,L,2,L,1,80000.0,120.0,4.0,1.0,1492.18,228.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,4,Row-major,Mix,transposed,transposed,yes,2,0,L,8,L,2,80000.0,120.0,4.0,1.0,1513.91,225.47
212,4,Row-major,Mix,transposed,no-transposed,no,2,0,L,8,L,2,80000.0,120.0,4.0,1.0,1528.85,223.27
213,4,Row-major,Mix,no-transposed,transposed,no,2,0,L,8,L,2,80000.0,120.0,4.0,1.0,3075.05,111.00
214,4,Row-major,Mix,no-transposed,transposed,yes,2,0,L,8,L,2,80000.0,120.0,4.0,1.0,3258.62,104.75


In [48]:
# 存储到数据库
df.to_sql('table1', conn, if_exists='append', index=False)

216

In [49]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM table1 ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

[(24,
  'Row-major',
  'Mix',
  'transposed',
  'transposed',
  'yes',
  '0',
  '0',
  'R',
  '2',
  'R',
  '2',
  79104.0,
  304.0,
  4.0,
  6.0,
  248.36,
  1328.7)]

插入24核数据

In [50]:
import os

directory_path = "/home/jiayh_lab/cse12012138/BenchmarkOptimization/logs/24cores/"  # 将此路径替换为您要搜索的目录的路径
file_names = [file for file in os.listdir(directory_path) if file.endswith(".out")]
data = []

for filename in file_names:
    print(filename)
    if 'fail' in filename: continue
    extract(directory_path + filename, data)

# 数据类型转换
columns_to_convert = range(11, 17)
data = [
    [float(value) if index in columns_to_convert else value for index, value in enumerate(row)]
    for row in data
]

# 加核数
data = [[24] + row for row in data]
df = pd.DataFrame(data, columns=['cores', 'PMAP', 'SWAP', 'L1', 'U', 'EQUIL', 'DEPTH', 'BCAST', 'RFACT', 'NDIV', 'PFACT', 'NBMIN', 'N', 'NB', 'P', 'Q', 'Time', 'Gflops'])
df

job.5890.out
job.6103.out
job.6113.out
job.6163.out
job.6200.out
job.6208.out
job.6251.out
job.6367.out
job.6497.out


,cores,PMAP,SWAP,L1,U,EQUIL,DEPTH,BCAST,RFACT,NDIV,PFACT,NBMIN,N,NB,P,Q,Time,Gflops
0,24,Row-major,Binary-exchange,transposed,transposed,no,0,0,L,2,L,1,80000.0,120.0,24.0,1.0,789.47,432.37
1,24,Row-major,Binary-exchange,transposed,transposed,yes,0,0,L,2,L,1,80000.0,120.0,24.0,1.0,797.81,427.85
2,24,Row-major,Binary-exchange,transposed,no-transposed,no,0,0,L,2,L,1,80000.0,120.0,24.0,1.0,806.87,423.05
3,24,Row-major,Binary-exchange,transposed,no-transposed,yes,0,0,L,2,L,1,80000.0,120.0,24.0,1.0,796.76,428.41
4,24,Row-major,Binary-exchange,no-transposed,transposed,no,0,0,L,2,L,1,80000.0,120.0,24.0,1.0,786.14,434.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,24,Row-major,Binary-exchange,no-transposed,transposed,yes,2,4,L,3,L,2,61000.0,260.0,24.0,1.0,362.37,417.60
3400,24,Row-major,Binary-exchange,no-transposed,no-transposed,no,2,4,L,3,L,2,61000.0,260.0,24.0,1.0,369.14,409.94
3401,24,Row-major,Binary-exchange,no-transposed,no-transposed,yes,2,4,L,3,L,2,61000.0,260.0,24.0,1.0,369.28,409.79
3402,24,Row-major,Spread-roll,transposed,transposed,no,2,4,L,3,L,2,61000.0,260.0,24.0,1.0,336.44,449.79


In [51]:
df.to_sql("table1", conn, if_exists="append", index=False)

3404

In [52]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM table1 ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

[(24,
  'Row-major',
  'Mix',
  'transposed',
  'transposed',
  'yes',
  '0',
  '0',
  'R',
  '2',
  'R',
  '2',
  79104.0,
  304.0,
  4.0,
  6.0,
  248.36,
  1328.7)]

In [53]:
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM table1 ORDER BY Gflops")

results = cursor.fetchall()
results

[(3976,)]

In [54]:
# 关闭游标和连接
cursor.close()

# 关闭数据库连接
conn.close()